## HOG + SVM

In [2]:
import os
import cv2
import numpy as np
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from sklearn.preprocessing import LabelEncoder
from tqdm import tqdm
from pycocotools.coco import COCO
from skimage.feature import hog
from skimage import exposure

# Path ke file anotasi COCO dan folder gambar
annotation_file = "../TrashClassFiltered/train/_annotations.coco.json"
image_dir = "../TrashClassFiltered/train/"
valid_annotation_file = "../TrashClassFiltered/valid/_annotations.coco.json"
valid_image_dir = "../TrashClassFiltered/valid/"

# Membaca dataset COCO
coco_train = COCO(annotation_file)
coco_valid = COCO(valid_annotation_file)

# Fungsi untuk memotong gambar berdasarkan bounding box
def crop_image(image, bbox):
    x, y, w, h = map(int, bbox)
    cropped_image = image[y:y+h, x:x+w]
    
    # Pastikan gambar yang dipotong memiliki ukuran yang memadai untuk HOG
    if cropped_image.shape[0] < 32 or cropped_image.shape[1] < 32:
        return None  # Jika terlalu kecil, kembalikan None
    
    return cropped_image


# Fungsi untuk ekstraksi fitur HOG
def extract_hog_features(image):
    if isinstance(image, str):
        image = cv2.imread(image)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    elif len(image.shape) == 3:  # Jika input adalah array gambar berwarna
        image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # Resize gambar agar ukuran konsisten
    image_resized = cv2.resize(image, (128, 128))
    
    # Pastikan gambar memiliki ukuran yang cukup untuk HOG
    if image_resized.shape[0] < 32 or image_resized.shape[1] < 32:
        return None
    
    # Ekstraksi fitur HOG
    fd, hog_image = hog(image_resized, orientations=9, pixels_per_cell=(8, 8), cells_per_block=(2, 2), visualize=True)
    
    hog_image_rescaled = exposure.rescale_intensity(hog_image, in_range=(0, 10))
    
    return fd

# Fungsi untuk memproses data pelatihan
def process_coco_data(coco, image_dir):
    features = []
    labels = []
    for image_id in tqdm(coco.getImgIds(), desc="Processing images"):
        image_info = coco.loadImgs(image_id)[0]
        annotations = coco.loadAnns(coco.getAnnIds(imgIds=image_id))
        image_path = os.path.join(image_dir, image_info['file_name'])
        image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
        if image is None:
            continue

        for ann in annotations:
            bbox = ann['bbox']
            category_id = ann['category_id']

            # Memotong gambar berdasarkan bounding box
            cropped_image = crop_image(image, bbox)
            if cropped_image is not None and cropped_image.size > 0:
                # Ekstraksi fitur HOG
                hog_features = extract_hog_features(cropped_image)
                if hog_features is not None:
                    features.append(hog_features)
                    labels.append(category_id)  # Tambahkan kategori ke daftar labels
    
    features = np.array(features)
    labels = np.array(labels)
    
    return features, labels

# Proses data pelatihan
print("Processing training data...")
X_train, y_train = process_coco_data(coco_train, image_dir)

loading annotations into memory...
Done (t=0.19s)
creating index...
index created!
loading annotations into memory...
Done (t=0.01s)
creating index...
index created!
Processing training data...


Processing images: 100%|██████████| 4485/4485 [02:28<00:00, 30.21it/s]


In [3]:
# Label encoding
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)

# Training classifier (SVM)
print("Training HOG + SVM...")
classifier = SVC(kernel='linear', probability=True)
classifier.fit(X_train, y_train_encoded)

Training HOG + SVM...


SVC(kernel='linear', probability=True)

In [4]:
# Proses data validasi untuk HOG
def process_validation_data(coco, image_dir):
    features = []
    labels = []  # Tetap menjadi list untuk menyimpan kategori
    for image_id in tqdm(coco.getImgIds(), desc="Processing validation images"):
        image_info = coco.loadImgs(image_id)[0]
        annotations = coco.loadAnns(coco.getAnnIds(imgIds=image_id))
        image_path = os.path.join(image_dir, image_info['file_name'])
        image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
        if image is None:
            continue

        for ann in annotations:
            bbox = ann['bbox']
            category_id = ann['category_id']

            # Memotong gambar berdasarkan bounding box
            cropped_image = crop_image(image, bbox)
            if cropped_image is not None and cropped_image.size > 0:
                # Ekstraksi fitur HOG
                hog_features = extract_hog_features(cropped_image)
                if hog_features is not None:
                    features.append(hog_features)
                    labels.append(category_id)  # Tambahkan kategori ke daftar labels
    
    features = np.array(features)
    labels = np.array(labels)
    
    return features, labels

# Proses data validasi untuk HOG
print("Processing validation data...")
X_valid, y_valid = process_validation_data(coco_valid, valid_image_dir)

# Label encoding untuk data validasi
y_valid_encoded = label_encoder.transform(y_valid)

# Prediksi menggunakan model SVM yang telah dilatih
y_pred = classifier.predict(X_valid)

# Menghitung akurasi
accuracy = accuracy_score(y_valid_encoded, y_pred)
print(f"Akurasi: {accuracy * 100:.2f}%")

# Menghitung precision, recall, dan F1-score
precision = precision_score(y_valid_encoded, y_pred, average='weighted', zero_division=1)
recall = recall_score(y_valid_encoded, y_pred, average='weighted', zero_division=1)
f1 = f1_score(y_valid_encoded, y_pred, average='weighted', zero_division=1)

print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1-Score: {f1:.2f}")

# Menampilkan confusion matrix
conf_matrix = confusion_matrix(y_valid_encoded, y_pred)
print("Confusion Matrix:")
print(conf_matrix)


Processing validation data...


Processing validation images: 100%|██████████| 1282/1282 [01:08<00:00, 18.82it/s]


Akurasi: 48.54%
Precision: 0.48
Recall: 0.49
F1-Score: 0.48
Confusion Matrix:
[[833 154 151 213]
 [258 471 102 141]
 [230 118 329 158]
 [265 139 122 302]]


## BoVW + SVM

In [ ]:
import os
import cv2
import numpy as np
from sklearn.cluster import KMeans
from sklearn.svm import SVC
from sklearn.metrics import precision_recall_curve, average_precision_score
from sklearn.preprocessing import LabelEncoder
from tqdm import tqdm
from pycocotools.coco import COCO

# Path ke file anotasi COCO dan folder gambar
annotation_file = "../TrashClassFiltered/train/_annotations.coco.json"
image_dir = "../TrashClassFiltered/train/"
valid_annotation_file = "../TrashClassFiltered/valid/_annotations.coco.json"
valid_image_dir = "../TrashClassFiltered/valid/"

# Membaca dataset COCO
coco_train = COCO(annotation_file)
coco_valid = COCO(valid_annotation_file)

# Fungsi untuk memotong gambar berdasarkan bounding box
def crop_image(image, bbox):
    x, y, w, h = map(int, bbox)
    return image[y:y+h, x:x+w]

# Fungsi untuk ekstraksi fitur SIFT
def extract_sift_features(image):
    if isinstance(image, str):
        image = cv2.imread(image)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    elif len(image.shape) == 3:  # Jika input adalah array gambar berwarna
        image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    sift = cv2.SIFT_create()
    keypoints, descriptors = sift.detectAndCompute(image, None)
    return descriptors

# Fungsi untuk membangun BoVW menggunakan KMeans
def build_bovw(descriptors_list, n_clusters=100):
    kmeans = KMeans(n_clusters=n_clusters)
    kmeans.fit(descriptors_list)
    return kmeans

# Fungsi untuk ekstraksi fitur BoVW
def extract_bovw_features(image_paths, kmeans):
    features = []
    valid_image_paths = []  # Menyimpan gambar yang berhasil diproses
    for image_path in tqdm(image_paths, desc="Extracting BoVW features"):
        descriptors = extract_sift_features(image_path)
        if descriptors is not None:
            # Representasikan gambar dengan histogram BoVW
            histogram = np.zeros(kmeans.n_clusters)
            labels = kmeans.predict(descriptors)
            for label in labels:
                histogram[label] += 1
            features.append(histogram)
            valid_image_paths.append(image_path)  # Menyimpan gambar yang berhasil diproses
    return np.array(features), valid_image_paths

# Proses data pelatihan untuk BoVW
def process_coco_data(coco, image_dir, kmeans):
    features = []
    labels = []  # Tetap menjadi list untuk menyimpan kategori
    for image_id in tqdm(coco.getImgIds(), desc="Processing images"):
        image_info = coco.loadImgs(image_id)[0]
        annotations = coco.loadAnns(coco.getAnnIds(imgIds=image_id))
        image_path = os.path.join(image_dir, image_info['file_name'])
        image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
        if image is None:
            continue

        for ann in annotations:
            bbox = ann['bbox']
            category_id = ann['category_id']

            # Memotong gambar berdasarkan bounding box
            cropped_image = crop_image(image, bbox)
            if cropped_image is not None and cropped_image.size > 0:
                descriptors = extract_sift_features(cropped_image)  # Ekstraksi fitur SIFT
                if descriptors is not None:
                    histogram = np.zeros(kmeans.n_clusters)
                    cluster_labels = kmeans.predict(descriptors)  # Prediksi cluster untuk setiap deskriptor
                    for label in cluster_labels:
                        histogram[label] += 1
                    features.append(histogram)
                    labels.append(category_id)  # Tambahkan kategori ke daftar labels
    return np.array(features), np.array(labels)

loading annotations into memory...
Done (t=1.00s)
creating index...
index created!
loading annotations into memory...
Done (t=0.02s)
creating index...
index created!


In [6]:
# Proses data pelatihan
print("Processing training data...")
train_image_paths = [os.path.join(image_dir, img) for img in os.listdir(image_dir) if img.endswith('.jpg')]
all_descriptors = []
for image_path in tqdm(train_image_paths, desc="Extracting descriptors"):
    descriptors = extract_sift_features(image_path)
    if descriptors is not None:
        all_descriptors.append(descriptors)
all_descriptors = np.vstack(all_descriptors)

Processing training data...


Extracting descriptors: 100%|██████████| 4485/4485 [03:18<00:00, 22.54it/s]


In [7]:
# Membangun BoVW menggunakan KMeans
kmeans = build_bovw(all_descriptors, n_clusters=100)

python(64525) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


In [8]:
# Proses data pelatihan untuk BoVW
X_train, y_train = process_coco_data(coco_train, image_dir, kmeans)

# Label encoding
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)

# Training classifier (SVM)
print("Training BoVW + SVM...")
classifier = SVC(kernel='linear', probability=True)
classifier.fit(X_train, y_train_encoded)

Processing images: 100%|██████████| 4485/4485 [04:34<00:00, 16.33it/s]


Training BoVW + SVM...


SVC(kernel='linear', probability=True)

In [9]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from sklearn.preprocessing import LabelEncoder

# Fungsi untuk memproses data validasi dan ekstraksi fitur BoVW
def process_validation_data(coco, image_dir, kmeans):
    features = []
    labels = []  # Tetap menjadi list untuk menyimpan kategori
    for image_id in tqdm(coco.getImgIds(), desc="Processing validation images"):
        image_info = coco.loadImgs(image_id)[0]
        annotations = coco.loadAnns(coco.getAnnIds(imgIds=image_id))
        image_path = os.path.join(image_dir, image_info['file_name'])
        image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
        if image is None:
            continue

        for ann in annotations:
            bbox = ann['bbox']
            category_id = ann['category_id']

            # Memotong gambar berdasarkan bounding box
            cropped_image = crop_image(image, bbox)
            if cropped_image is not None and cropped_image.size > 0:
                descriptors = extract_sift_features(cropped_image)  # Ekstraksi fitur SIFT
                if descriptors is not None:
                    histogram = np.zeros(kmeans.n_clusters)
                    cluster_labels = kmeans.predict(descriptors)  # Prediksi cluster untuk setiap deskriptor
                    for label in cluster_labels:
                        histogram[label] += 1
                    features.append(histogram)
                    labels.append(category_id)  # Tambahkan kategori ke daftar labels
    return np.array(features), np.array(labels)

# Proses data validasi untuk BoVW
print("Processing validation data...")
valid_image_paths = [os.path.join(valid_image_dir, img) for img in os.listdir(valid_image_dir) if img.endswith('.jpg')]
all_valid_descriptors = []
for image_path in tqdm(valid_image_paths, desc="Extracting validation descriptors"):
    descriptors = extract_sift_features(image_path)
    if descriptors is not None:
        all_valid_descriptors.append(descriptors)
all_valid_descriptors = np.vstack(all_valid_descriptors)

# Proses data validasi untuk BoVW
X_valid, y_valid = process_validation_data(coco_valid, valid_image_dir, kmeans)

# Label encoding untuk data validasi
y_valid_encoded = label_encoder.transform(y_valid)

# Prediksi menggunakan model SVM yang telah dilatih
y_pred = classifier.predict(X_valid)

# Menghitung akurasi
accuracy = accuracy_score(y_valid_encoded, y_pred)
print(f"Akurasi: {accuracy * 100:.2f}%")

# Menghitung precision, recall, dan F1-score
precision = precision_score(y_valid_encoded, y_pred, average='weighted', zero_division=1)
recall = recall_score(y_valid_encoded, y_pred, average='weighted', zero_division=1)
f1 = f1_score(y_valid_encoded, y_pred, average='weighted', zero_division=1)

print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1-Score: {f1:.2f}")

# Menampilkan confusion matrix
conf_matrix = confusion_matrix(y_valid_encoded, y_pred)
print("Confusion Matrix:")
print(conf_matrix)


Processing validation data...


Processing validation images: 100%|██████████| 1282/1282 [01:53<00:00, 11.32it/s]


Akurasi: 46.19%
Precision: 0.56
Recall: 0.46
F1-Score: 0.42
Confusion Matrix:
[[1288   73   28   71]
 [ 778  265   36   33]
 [ 473   38  312   62]
 [ 702   12   30  140]]
